# a. 尝试使用 LoRA 微调 Stable Diffusion 模型（文生图）

> [HW10: Stable Diffusion Fine-tuning](https://colab.research.google.com/drive/1dI_-HVggxyIwDVoreymviwg6ZOvEHiLS?usp=sharing#scrollTo=CnJtiRaRuTFX) 中文镜像版
>
> 这次不是1:1的镜像，我重构了主要的代码逻辑，以便于阅读。对于可以访问 Colab 的同学来说，一样建议使用下面这份代码进行学习。
>
> 修改：
> 1. Colab 中的代码每次训练时先导入了 checkpoint-last 文件夹下的 unet 和 text_encoder，这就意味着每次重新运行是强制 resume 的。
> 我对这个行为进行了修改，你现在可以通过指定 prepare_lora_model 中的 resume 参数来选择是否接着之前的进行训练。
> 建议设置为 False，每次重新运行才能对不同参数有直观的感受。
>
> 2. 原代码中导入了 PEFT 并设置了 LoRA 参数却没有用到，出于学习而非提交作业的目的，我会重新写这一模块。
> 重写之后的可训练参数层与数量已经经过验证，与 [yahcreeper/GenAI-HW10-Model](https://huggingface.co/yahcreeper/GenAI-HW10-Model/tree/main/checkpoint-last) 完全一致，可放心使用当前版本（关于可训练数量，可以参考[《14. PEFT：在大模型中快速应用 LoRA.md》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/14.%20PEFT：在大模型中快速应用%20LoRA.md)使用 `print_trainable_parameters()` 进行打印）。
>
> 3. 会增加注释以便理解。
>
> 指导文章：[16. 用 LoRA 微调 Stable Diffusion：拆开炼丹炉，动手实现你的第一次 AI 绘画](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/16.%20用%20LoRA%20微调%20Stable%20Diffusion：拆开炼丹炉，动手实现你的第一次%20AI%20绘画.md)
> 

你将微调你自己的 Stable Diffusion 模型，从给定的文本描述生成自定义图像，在初始化项目之后，你将可以在 `SD/Datasets/Brad` 下看到图像和对应的文本描述。

当前代码使用到的数据集 `dataset.zip` 已经放在 Demos/data/14 下面。

你需要注意的是，这份代码倾向于如何微调而非加载数据集或者评估，对于初学者，可以仅关注训练参数和LoRA部分，其余部分的代码细节不用关注。

这次的学习可以在两种版本中二选一，学习效果一致。

[版本 b - 精简版](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/14b.%20尝试使用%20LoRA%20微调%20Stable%20Diffusion%20模型%20-%20精简版.ipynb)

这里还有一个简单的 [🎡 SD LoRA 脚本](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/sd_lora.py)供你尝试，详见：[CodePlayground](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/README.md#当前的玩具)，点击 `►` 或对应的文本展开。

在线链接（精简版）：[Kaggle](https://www.kaggle.com/code/aidemos/14b-lora-stable-diffusion) | [Colab](https://colab.research.google.com/drive/1idmnaQZwRhjUPw7ToEXlVo82Mihfl_aA?usp=sharing)

## 安装必要的库

本单元中将安装一些库用于微调。

安装大约需要 5 分钟。

In [ ]:
!pip install timm
!pip install fairscale
!pip install transformers
!pip install requests
!pip install accelerate
!pip install diffusers
!pip install einop
!pip install safetensors
!pip install voluptuous
!pip install jax
!pip install jaxlib
!pip install peft
!pip install deepface==0.0.90
!pip install tensorflow==2.9.0  # 为了避免最后评估阶段使用deepface时的错误，这里选择降级版本
!pip install keras
!pip install opencv-python
#!pip install gdown

## 导入

In [ ]:
# ========== 标准库模块 ==========
import os
import math
import random
import glob
import shutil
import argparse
from pathlib import Path
import subprocess

# ========== 第三方库 ==========
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from tqdm.auto import tqdm

# ========== 深度学习相关库 ==========
from torchvision import transforms
from torchvision.utils import save_image

# Transformers (Hugging Face)
from transformers import AutoProcessor, AutoModel, CLIPTextModel, CLIPTokenizer

# Diffusers (Hugging Face)
from diffusers import (
    AutoencoderKL,
    DDPMScheduler,
    DiffusionPipeline,
    StableDiffusionPipeline,
    UNet2DConditionModel
)
from diffusers.optimization import get_scheduler
from diffusers.utils import convert_state_dict_to_diffusers
from diffusers.training_utils import compute_snr
from diffusers.utils.torch_utils import is_compiled_module

# ========== LoRA 模型库 ==========
from peft import LoraConfig, get_peft_model

# ========== 面部检测库 ==========
from deepface import DeepFace

# ========== IPython 和 Widgets ==========
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

import cv2

## 准备项目

直接运行代码，这个模块不用修改参数，不用关心这里的代码细节，除非你对交互感兴趣。

注意：记得点击**初始化项目**

当你看见✅时，代表项目已经准备好。

In [ ]:
# 创建项目名称输入框
project_name_widget = widgets.Text(
    value="Brad",
    description="项目名称:",
    style={'description_width': 'initial'}
)

# 创建数据集名称输入框
dataset_name_widget = widgets.Text(
    value="Brad",  # "Brad-512", "Anne-512"
    description="数据集:",
    style={'description_width': 'initial'},
    disabled=True
)

# 创建提交按钮
submit_button = widgets.Button(description="初始化项目")

# 输出区域
output_area = widgets.Output()

# 定义项目初始化逻辑
def initialize_project(b):
    global project_name, dataset_name, root_dir, main_dir, project_dir, model_path, images_folder, prompts_folder, captions_folder
    with output_area:
        clear_output()
        project_name = project_name_widget.value.strip()
        dataset_name = dataset_name_widget.value.strip()
        
        if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
            print("请输入有效的项目名称。")
        else:
            # 创建项目目录
            project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
            project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]
            root_dir = "./"  # 当前目录
            main_dir = os.path.join(root_dir, "SD")  # 主目录
            project_dir = os.path.join(main_dir, project_name)  # 项目目录
            
            # 确保目录存在
            os.makedirs(main_dir, exist_ok=True)
            os.makedirs(project_dir, exist_ok=True)
            
            # 定义数据集和模型路径
            zip_file = os.path.join("./", "data/14/Datasets.zip")
            model_path = os.path.join(project_dir, "logs", "checkpoint-last")
            images_folder = os.path.join(main_dir, "Datasets", dataset_name)
            prompts_folder = os.path.join(main_dir, "Datasets", "prompts")
            captions_folder = images_folder

            # 检查并解压数据集
            print("📂 正在检查并解压样例数据集...")
            
            if not os.path.exists(zip_file):
                print("❌ 未找到数据集压缩文件 Datasets.zip！")
                print("请下载数据集:\nhttps://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/data/14/Datasets.zip\n并放在 ./data/14 文件夹下")
            else:
                subprocess.run(f"unzip -q -o {zip_file} -d {main_dir}", shell=True)
                
                # # 克隆模型文件
                print("📂 默认不克隆LoRA模型文件而是自己训练，因为下载需要占用2G且等价于你先进行1000个step训练。\n")
    
                # 创建必要的文件夹
                os.makedirs(images_folder, exist_ok=True)
    
                print(f"✅ 项目 {project_name} 已准备好！")

# 将提交按钮与项目初始化逻辑绑定
submit_button.on_click(initialize_project)

# 显示控件
display(project_name_widget, dataset_name_widget, submit_button, output_area)

## 设置参数

In [ ]:
# 请勿随意更改以下参数，除非你自己知道改的参数对应的什么，否则可能因 GPU 内存不足导致进程崩溃。
output_folder = os.path.join(project_dir, "logs")  # 存放 model checkpoints 和 validation 的文件夹
seed = 1126  # 随机数种子
train_batch_size = 2  # 训练批次大小，即每次训练中处理的样本数量
resolution = 512  # 训练图像的分辨率
weight_dtype = torch.bfloat16  # 权重数据类型，使用 bfloat16 以节省内存并加快计算速度
snr_gamma = 5  # SNR 参数，用于信噪比加权损失的调节系数

### Stable Diffusion LoRA 的微调参数

In [ ]:
# 预训练的 Stable Diffusion 模型路径，用于加载模型进行微调
pretrained_model_name_or_path = "stablediffusionapi/cyberrealistic-41"

# LoRA 配置
lora_config = LoraConfig(
    r=32,  # LoRA 的秩，即低秩矩阵的维度，决定了参数调整的自由度
    lora_alpha=16,  # 缩放系数，控制 LoRA 权重对模型的影响
    target_modules=[
        "q_proj", "v_proj", "k_proj", "out_proj",  # 指定 Text encoder 的 LoRA 应用对象（用于调整注意力机制中的投影矩阵）
        "to_k", "to_q", "to_v", "to_out.0"  # 指定 UNet 的 LoRA 应用对象（用于调整 UNet 中的注意力机制）
    ],
    lora_dropout=0  # LoRA dropout 概率，0 表示不使用 dropout
)

### 其他设置

In [ ]:
# 学习率调度器部分
lr_scheduler_name = "cosine_with_restarts" # 设置学习率的调度器
lr_warmup_steps = 100 # 设置学习率预热步数

# prompt 处理
validation_prompt_name = "validation_prompt.txt"
validation_prompt_path = os.path.join(prompts_folder, validation_prompt_name)
with open(validation_prompt_path, "r") as f:
    validation_prompt = [line.strip() for line in f.readlines()]

直接运行代码，交互式修改参数，不用关心这里的代码细节，除非你对交互感兴趣。

In [ ]:
# 学习率部分
learning_rate_markdown = Markdown("▶️ **Learning Rate**<br>学习率是影响结果最重要的参数，当前默认三者一致。<br>如果你想慢速训练且有大量图像，或者如果 LoRA rank（有些文生图UI可能用的是dim） 和 alpha 值较高，建议将 unet 的学习率调至 5e-5 或更低。<br>需要注意的是，实际上learning_rate在这里不起作用，我们只使用对应模块的学习率。<br>这里进行展示是为了让你之后在炼丹炉上能够更好的理解设置。")
learning_rate_widget = widgets.FloatText(
    value=1e-4,
    description='learning_rate:',
    style={'description_width': '200px'},
    layout=widgets.Layout(width='300px'),
    disabled=True  # 禁用手动修改
)

unet_learning_rate_widget = widgets.FloatText(
    value=1e-4,
    description='unet_learning_rate:',
    style={'description_width': '200px'},
    layout=widgets.Layout(width='300px')
)

text_encoder_learning_rate_widget = widgets.FloatText(
    value=1e-4,
    description='text_encoder_learning_rate:',
    style={'description_width': '200px'},
    layout=widgets.Layout(width='300px')
)

# 训练步骤部分
max_train_steps_markdown = Markdown("▶️ **Steps**<br>选择训练步骤和每次验证生成的图像数量")
max_train_steps_widget = widgets.IntSlider(
    value=200,
    min=200,
    max=2000,
    step=100,
    description='max_train_steps:',
    continuous_update=False,
    style={'description_width': '200px'},
    layout=widgets.Layout(width='400px')
)


validation_prompt_num_widget = widgets.IntSlider(
    value=3,
    min=1,
    max=5,
    step=1,
    description='validation_prompt_num:',
    style={'description_width': '200px'},
    layout=widgets.Layout(width='400px')
)


validation_step_ratio_widget = widgets.FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    description='validation_step_ratio:',
    style={'description_width': '200px'},
    layout=widgets.Layout(width='400px')
)


validation_prompt_widget = widgets.Text(
    value = validation_prompt_path,
    description='validation_prompt_path:',
    style={'description_width': '200px'},
    layout=widgets.Layout(width='550px'),
    disabled=True  # 禁用手动修改
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并打印配置
def on_button_click(b):
    global unet_learning_rate, text_encoder_learning_rate, max_train_steps, validation_prompt, validation_prompt_num, validation_step_ratio
    with output_area:  # 使用 Output 小部件捕获输出
        output_area.clear_output()  # 清除之前的输出
        unet_learning_rate = unet_learning_rate_widget.value
        text_encoder_learning_rate = text_encoder_learning_rate_widget.value
        max_train_steps = max_train_steps_widget.value
        validation_prompt_num = validation_prompt_num_widget.value
        validation_step_ratio = validation_step_ratio_widget.value

        # 打印配置
        print(f"UNet 学习率: {unet_learning_rate}")
        print(f"文本编码器学习率: {text_encoder_learning_rate}")
        print(f"训练步骤: {max_train_steps}")
        print(f"验证prompt数量: {validation_prompt_num}")
        print(f"验证步骤比例: {validation_step_ratio}")
        print(f"用于验证的prompt文件位置: {validation_prompt_path}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(learning_rate_markdown, learning_rate_widget, unet_learning_rate_widget, text_encoder_learning_rate_widget)
display(max_train_steps_markdown, max_train_steps_widget, validation_prompt_num_widget, validation_step_ratio_widget, validation_prompt_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)


## 定义一些有用的函数和类


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_EXTENSIONS = [".png", ".jpg", ".jpeg", ".webp", ".bmp", ".PNG", ".JPG", ".JPEG", ".WEBP", ".BMP"]

# 数据增强
train_transform = transforms.Compose(
    [
        transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),  # 调整图像大小
        transforms.CenterCrop(resolution),  # 中心裁剪图像
        transforms.RandomHorizontalFlip(),  # 随机水平翻转
        transforms.ToTensor(),  # 将图像转换为张量
    ]
)
class Text2ImageDataset(torch.utils.data.Dataset):
    """
    (1) 目标:
        - 用于构建文本到图像模型的微调数据集

    """
    def __init__(self, images_folder, captions_folder, transform, tokenizer):
        """
        (2) 参数:
            - images_folder: str, 图像文件夹路径
            - captions_folder: str, 标注文件夹路径
            - transform: function, 将原始图像转换为 torch.tensor
            - tokenizer: CLIPTokenizer, 将文本标注转为 word ids
        """
        # 初始化图像路径列表，并根据指定的扩展名找到所有图像文件
        self.image_paths = []
        for ext in IMAGE_EXTENSIONS:
            self.image_paths.extend(glob.glob(f"{images_folder}/*{ext}"))
        self.image_paths = sorted(self.image_paths)

        # 通过 DeepFace 库提取每张图像的面部嵌入（特征向量）
        self.train_emb = torch.tensor([DeepFace.represent(img_path, detector_backend="ssd", model_name="GhostFaceNet", enforce_detection=False)[0]['embedding'] for img_path in self.image_paths])

        # 加载对应的文本标注，依次读取每个文本文件中的内容
        caption_paths = sorted(glob.glob(f"{captions_folder}/*txt"))
        captions = []
        for p in caption_paths:
            with open(p, "r") as f:
                captions.append(f.readline())

        # 使用 tokenizer 将文本标注转换为 word ids
        inputs = tokenizer(
            captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        self.input_ids = inputs.input_ids
        self.transform = transform

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        input_id = self.input_ids[idx]
        try:
            # 加载图像并将其转换为 RGB 模式，然后应用数据增强
            image = Image.open(img_path).convert("RGB")
            tensor = self.transform(image)
        except Exception as e:
            print(f"无法加载图像路径: {img_path}, 错误: {e}")
            return None

        return tensor, input_id  # 返回处理后的图像和相应的文本标注

    def __len__(self):
        return len(self.image_paths)
    
def prepare_lora_model(lora_config, pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5", model_path=None, resume=False):
    """
    (1) 目标:
        - 用于加载完整的 Stable Diffusion 模型，包括 Lora 层，并冻结非 Lora 参数。这包括 Tokenizer、噪声调度器、UNet、VAE 和 文本编码器。

    (2) 参数:
        - pretrained_model_name_or_path: str, 从 Hugging Face 获取的模型名称
        - model_path: str, 预训练模型的路径
        - resume: bool, 是否从上一次训练中恢复

    (3) 返回:
        - 输出: Tokenizer, 噪声调度器, UNet, VAE, 文本编码器

    """
    # 加载噪声调度器，用于控制扩散模型的噪声添加和移除过程
    noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")

    # 加载 Tokenizer，用于将文本标注转换为 tokens
    tokenizer = CLIPTokenizer.from_pretrained(
        pretrained_model_name_or_path,
        subfolder="tokenizer"
    )

    # 加载 CLIP 文本编码器，用于将文本标注转换为特征向量
    text_encoder = CLIPTextModel.from_pretrained(
        pretrained_model_name_or_path,
        torch_dtype=weight_dtype,
        subfolder="text_encoder"
    )

    # 加载 VAE 模型，用于在扩散模型中处理图像的潜在表示
    vae = AutoencoderKL.from_pretrained(
        pretrained_model_name_or_path,
        subfolder="vae"
    )

    # 加载 UNet 模型，负责处理扩散模型中的图像生成和推理过程
    unet = UNet2DConditionModel.from_pretrained(
        pretrained_model_name_or_path,
        torch_dtype=weight_dtype,
        subfolder="unet"
    )

    # 将 LoRA 配置应用到 text_encoder 和 unet
    text_encoder = get_peft_model(text_encoder, lora_config)
    unet = get_peft_model(unet, lora_config)
    
    # 打印可训练参数数量
    text_encoder.print_trainable_parameters()
    unet.print_trainable_parameters()
    
    # 如果设置为继续训练，则加载上一次的模型权重，当然，你可以修改 model_path 来指定其他的路径
    if resume:
        if model_path is None or not os.path.exists(model_path):
            raise ValueError("当resume设置为True的时候你必须提供有效的model_path")
            
        # 加载上次训练的模型权重，注意这里只加载权重，而不是覆盖整个模型，覆盖：model = torch.load(...)
        text_encoder = torch.load(os.path.join(model_path, "text_encoder.pt"))
        unet = torch.load(os.path.join(model_path, "unet.pt"))
    
    # 冻结 VAE 参数
    vae.requires_grad_(False)

    # 使用 get_peft_model() 后会自动冻结其中的非 LoRA 层，所以可以注释掉
    # # 冻结 UNet 和文本编码器中的非 Lora 参数
    # for name, param in unet.named_parameters():
    #     if "lora" in name:
    #         param.requires_grad_(True)
    #     else:
    #         param.requires_grad_(False)
    # for name, param in text_encoder.named_parameters():
    #     if "lora" in name:
    #         param.requires_grad_(True)
    #     else:
    #         param.requires_grad_(False)

    # 将模型移动到 GPU 上并设置权重的数据类型
    unet.to(DEVICE, dtype=weight_dtype)
    vae.to(DEVICE, dtype=weight_dtype)
    text_encoder.to(DEVICE, dtype=weight_dtype)
    
    return tokenizer, noise_scheduler, unet, vae, text_encoder


def prepare_optimizer(unet, text_encoder, unet_learning_rate=5e-4, text_encoder_learning_rate=1e-4):
    """
    (1) 目标:
        - 为 UNet 和文本编码器的可训练参数分别设置优化器，并指定不同的学习率。

    (2) 参数:
        - unet: UNet2DConditionModel, Hugging Face 的 UNet 模型
        - text_encoder: CLIPTextModel, Hugging Face 的文本编码器
        - unet_learning_rate: float, UNet 的学习率
        - text_encoder_learning_rate: float, 文本编码器的学习率

    (3) 返回:
        - 输出: 优化器 Optimizer
    """
    # 筛选出 UNet 中需要训练的 Lora 层参数
    unet_lora_layers = [p for p in unet.parameters() if p.requires_grad]
    
    # 筛选出文本编码器中需要训练的 Lora 层参数
    text_encoder_lora_layers = [p for p in text_encoder.parameters() if p.requires_grad]
    
    # 将需要训练的参数分组并设置不同的学习率
    trainable_params = [
        {"params": unet_lora_layers, "lr": unet_learning_rate},
        {"params": text_encoder_lora_layers, "lr": text_encoder_learning_rate}
    ]
    
    # 使用 AdamW 优化器
    optimizer = torch.optim.AdamW(trainable_params)
    
    return optimizer


def evaluate(pretrained_model_name_or_path, weight_dtype, seed, unet_path, text_encoder_path, validation_prompt, output_folder, train_emb):
    """
    (1) 目标:
        - 用于加载给定路径中的 UNet 和文本编码器模型，生成图像并计算面部相似性、CLIP 分数和无面部图像的数量。

    (2) 参数:
        - pretrained_model_name_or_path: str, Hugging Face 的模型名称
        - weight_dtype: torch.dtype, 模型的权重数据类型
        - seed: int, 随机数种子
        - unet_path: str, UNet 模型检查点路径
        - text_encoder_path: str, 文本编码器模型检查点路径
        - validation_prompt: list, 存储验证文本的字符串列表
        - output_folder: str, 保存生成图像的文件夹
        - train_emb: tensor, 训练图像的面部特征

    (3) 返回:
        - face_score: float, 面部相似性评分
        - clip_score: float, CLIP 分数
        - mis: int, 无法检测到面部的图像数量

    """
    # 加载 DiffusionPipeline 并设置 UNet 和文本编码器
    pipeline = DiffusionPipeline.from_pretrained(
        pretrained_model_name_or_path,
        torch_dtype=weight_dtype,
        safety_checker=None,
    )
    pipeline.unet = torch.load(unet_path)  # 加载 UNet 模型
    pipeline.text_encoder = torch.load(text_encoder_path)  # 加载文本编码器模型
    pipeline = pipeline.to(DEVICE)

    # 加载 CLIP 模型用于计算图像与文本的相似性分数
    clip_model_name = "openai/clip-vit-base-patch32"
    clip_model = AutoModel.from_pretrained(clip_model_name).to(DEVICE)
    clip_processor = AutoProcessor.from_pretrained(clip_model_name)
    
    # 推理生成图像
    with torch.no_grad():
        generator = torch.Generator(device=DEVICE)
        generator.manual_seed(seed)
        face_score = 0
        clip_score = 0
        mis = 0  # 记录无法检测到面部的图像数量
        print("正在生成验证图像......")
        images = []
        for i in range(0, len(validation_prompt), 4):
            images.extend(pipeline(validation_prompt[i:min(i + 4, len(validation_prompt))], num_inference_steps=30, generator=generator).images)
        
        # 计算面部相似性和 CLIP 分数
        print("正在计算验证分数......")
        valid_emb = []
        for i, image in enumerate(tqdm(images)):
            save_file = f"{output_folder}/valid_image_{i}.png"
            image.save(save_file)  # 将生成的图像保存到指定目录
            opencvImage = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

            # 使用 DeepFace 检测面部特征
            emb = DeepFace.represent(
                opencvImage,
                detector_backend="ssd",
                model_name="GhostFaceNet",
                enforce_detection=False,
            )
            # 如果无法检测到面部，计入 mis
            if emb == [] or emb[0]['face_confidence'] == 0:
                mis += 1
                continue

            # 计算 CLIP 分数，衡量图像与文本的相似度
            emb = emb[0]
            inputs = clip_processor(text=validation_prompt[i], images=image, return_tensors="pt").to(DEVICE)
            with torch.no_grad():
                outputs = clip_model(**inputs)
            sim = outputs.logits_per_image  # 获取图像与文本的相似性得分
            clip_score += sim.item()
            valid_emb.append(emb['embedding'])

        # 如果生成的图片都没有人脸，直接 0 分
        if len(valid_emb) == 0:
            return 0, 0, mis
        
        # 计算面部相似性分数
        valid_emb = torch.tensor(valid_emb)
        valid_emb = (valid_emb / torch.norm(valid_emb, p=2, dim=-1)[:, None]).cuda()
        train_emb = (train_emb / torch.norm(train_emb, p=2, dim=-1)[:, None]).cuda()
        face_score = torch.cdist(valid_emb, train_emb, p=2).mean().item()
        clip_score /= len(validation_prompt) - mis
    
    return face_score, clip_score, mis   # 返回面部相似性、CLIP 分数和无面部图像的数量

## 准备微调所需的模型和优化器


In [ ]:
# 决定是否继续之前的训练
resume = False

# 准备微调所需的 tokenizer, 噪声调度器, UNet, VAE 和文本编码器
tokenizer, noise_scheduler, unet, vae, text_encoder = prepare_lora_model(lora_config, pretrained_model_name_or_path, model_path, resume)

# 准备优化器，用于更新 UNet 和 文本编码器中经过 LoRA 层调整的可训练参数
optimizer = prepare_optimizer(unet, text_encoder, unet_learning_rate, text_encoder_learning_rate)

# 设置学习率调度器，控制学习率的变化
# get_scheduler 根据指定的调度器类型（例如线性、余弦等），为优化器提供一个动态调整学习率的策略
# num_warmup_steps: 在初始阶段进行学习率预热的步数
# num_training_steps: 总的训练步数
# num_cycles: 用于余弦调度器，指定余弦衰减周期的数量
lr_scheduler = get_scheduler(
    lr_scheduler_name,
    optimizer=optimizer,
    num_warmup_steps=lr_warmup_steps,
    num_training_steps=max_train_steps,
    num_cycles=3
)

# 准备数据集
# Text2ImageDataset 会根据指定的图像文件夹和文本标注文件夹，加载并处理数据
# 具体：使用指定的 transform 来预处理图像数据，并使用 tokenizer 来将文本转换为 tokens
dataset = Text2ImageDataset(
    images_folder=images_folder,
    captions_folder=captions_folder,
    transform=train_transform,
    tokenizer=tokenizer,
)

# 设置数据加载器，负责将数据集分批次加载到模型中进行训练
# collate_fn: 自定义的批处理函数，将数据集中每个样本的图像和文本标注合并成批次
def collate_fn(examples):
    pixel_values = []
    input_ids = []
    
    # 将每个样本中的图像 tensor 和文本标注 input_ids 分别收集到列表中
    for tensor, input_id in examples:
        pixel_values.append(tensor)  # 将图像张量收集到 pixel_values 列表中
        input_ids.append(input_id)  # 将文本标注收集到 input_ids 列表中
    
    # 将列表中的所有图像张量堆叠成一个大的 batch tensor
    pixel_values = torch.stack(pixel_values, dim=0).float()  # 形状为 (batch_size, channels, height, width)
    input_ids = torch.stack(input_ids, dim=0)  # 形状为 (batch_size, max_seq_length)
    
    # 返回批处理后的图像和文本标注
    return {"pixel_values": pixel_values, "input_ids": input_ids}

# 使用 PyTorch DataLoader 加载数据集，设置每个批次的大小和线程数
train_dataloader = torch.utils.data.DataLoader(
    dataset,
    shuffle=True,  # 打乱数据
    collate_fn=collate_fn,  # 自定义批处理函数
    batch_size=train_batch_size,  # 每个批次的样本数量
    num_workers=8,  # 使用 8 个线程并行加载数据，加快数据加载速度
)

print("准备完成！")

## 开始微调

最后是开始进行微调的部分：


In [ ]:
# 开始微调 Stable Diffusion 模型
os.environ["TOKENIZERS_PARALLELISM"] = "false"
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# 进度条显示训练进度
progress_bar = tqdm(
    range(0, max_train_steps),
    initial=0,
    desc="步骤",
)

# 初始化全局步数和其他控制变量
global_step = 0
num_epochs = math.ceil(max_train_steps / len(train_dataloader))  # 根据最大训练步数计算总共的 epoch 数量
validation_step = int(max_train_steps * validation_step_ratio)  # 根据设置的验证步数比例计算每隔多少步进行验证
best_face_score = float("inf")  # 初始化为正无穷大，存储最佳面部相似度分数

# 训练循环，遍历所有的 epoch
for epoch in range(num_epochs):
    # 将模型切换到训练模式
    unet.train()
    text_encoder.train()
    
    # 遍历数据加载器，进行逐步训练
    for step, batch in enumerate(train_dataloader):
        if global_step >= max_train_steps:
            break  # 当达到最大训练步数时，终止训练
        
        # 编码图像为潜在表示（latent），通过 VAE 对图像进行编码
        latents = vae.encode(batch["pixel_values"].to(DEVICE, dtype=weight_dtype)).latent_dist.sample()
        latents = latents * vae.config.scaling_factor  # 根据 VAE 的缩放因子调整潜在空间

        # 为潜在表示添加噪声，生成带噪声的图像
        noise = torch.randn_like(latents)  # 生成与潜在表示相同形状的随机噪声
        bsz = latents.shape[0]  # 批次大小
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device).long()  # 随机选择时间步
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)  # 添加噪声到潜在表示

        # 获取文本的嵌入表示
        encoder_hidden_states = text_encoder(batch["input_ids"].to(latents.device), return_dict=False)[0]
        
        # 计算目标值，基于预测类型（epsilon 或 v_prediction）
        if noise_scheduler.config.prediction_type == "epsilon":
            target = noise  # 预测噪声
        elif noise_scheduler.config.prediction_type == "v_prediction":
            target = noise_scheduler.get_velocity(latents, noise, timesteps)  # 预测速度向量

        # UNet 模型预测，输入噪声潜在空间、时间步和文本嵌入
        model_pred = unet(noisy_latents, timesteps, encoder_hidden_states, return_dict=False)[0]
        
        # 计算损失，基于是否设置了 snr_gamma，这个参数在最开始的部分进行了设置
        if not snr_gamma:
            loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")  # 使用均方误差 (MSE) 作为损失
        else:
            # 计算信噪比 (SNR) 并根据 SNR 加权 MSE 损失
            snr = compute_snr(noise_scheduler, timesteps)
            mse_loss_weights = torch.stack([snr, snr_gamma * torch.ones_like(timesteps)], dim=1).min(dim=1)[0]
            if noise_scheduler.config.prediction_type == "epsilon":
                mse_loss_weights = mse_loss_weights / snr
            elif noise_scheduler.config.prediction_type == "v_prediction":
                mse_loss_weights = mse_loss_weights / (snr + 1)
            
            # 计算加权的 MSE 损失
            loss = F.mse_loss(model_pred.float(), target.float(), reduction="none")
            loss = loss.mean(dim=list(range(1, len(loss.shape)))) * mse_loss_weights
            loss = loss.mean()

        # 反向传播
        loss.backward()  # 计算梯度
        optimizer.step()  # 更新模型参数
        lr_scheduler.step()  # 更新学习率
        optimizer.zero_grad()  # 清零梯度
        progress_bar.update(1)  # 更新进度条
        global_step += 1  # 更新全局步数


        # 验证模型性能
        if global_step % validation_step == 0 or global_step == max_train_steps:
            # 保存当前检查点（包含 UNet 和 文本编码器的模型参数）
            save_path = os.path.join(output_folder, f"checkpoint-last")
            unet_path = os.path.join(save_path, "unet.pt")
            text_encoder_path = os.path.join(save_path, "text_encoder.pt")
            print(f"正在保存检查点到 {save_path} ......")
            os.makedirs(save_path, exist_ok=True)
            torch.save(unet, unet_path)
            torch.save(text_encoder, text_encoder_path)
            
            # 进行验证
            save_path = os.path.join(output_folder, f"checkpoint-{global_step}")
            os.makedirs(save_path, exist_ok=True)
            face_score, clip_score, mis = evaluate(
                pretrained_model_name_or_path=pretrained_model_name_or_path,
                weight_dtype=weight_dtype,
                seed=seed,
                unet_path=unet_path,
                text_encoder_path=text_encoder_path,
                validation_prompt=validation_prompt[:validation_prompt_num],  # 验证提示文本
                output_folder=save_path,  # 保存验证结果的文件夹
                train_emb=dataset.train_emb  # 训练数据中的面部特征嵌入
            )
            print("步骤:", global_step, "面部相似度评分:", face_score, "CLIP评分:", clip_score, "无面部图像数量:", mis)
            
            # 如果当前面部相似度评分优于之前的最佳记录，保存最佳模型
            if face_score < best_face_score:
                best_face_score = face_score
                save_path = os.path.join(output_folder, f"checkpoint-best")
                unet_path = os.path.join(save_path, "unet.pt")
                text_encoder_path = os.path.join(save_path, "text_encoder.pt")
                os.makedirs(save_path, exist_ok=True)
                torch.save(unet, unet_path)  # 保存最佳的 UNet 模型
                torch.save(text_encoder, text_encoder_path)  # 保存最佳的文本编码器模型

print("微调完成！")

## 生成图像和测试

微调过程已经完成。接下来将测试模型。

首先加载之前保存的微调模型的检查点，并计算面部相似度、CLIP 评分以及没有面部的图像数量。

In [ ]:
# 设置用于推理的 checkpoint 路径
checkpoint_path = os.path.join(output_folder, f"checkpoint-last")  # 设置推理时使用最后保存的 checkpoint

# 设置路径
unet_path = os.path.join(checkpoint_path, "unet.pt")
text_encoder_path = os.path.join(checkpoint_path, "text_encoder.pt")
inference_path = os.path.join(project_dir, "inference")
os.makedirs(inference_path, exist_ok=True)

# 获取训练图像的路径
train_image_paths = []  # 初始化存储所有训练图像路径的列表
for ext in IMAGE_EXTENSIONS:
    # 通过扩展名遍历图像文件夹中的所有图像
    train_image_paths.extend(glob.glob(f"{images_folder}/*{ext}"))
train_image_paths = sorted(train_image_paths)  # 对图像路径进行排序

# 初始化列表存储面部特征嵌入
train_emb_list = []

# 遍历训练图像路径并提取面部特征嵌入
for img_path in train_image_paths:
    # 使用 DeepFace 从每张图像中提取面部特征
    face_representation = DeepFace.represent(
        img_path, 
        detector_backend="ssd",  # 使用 ssd 检测器
        model_name="GhostFaceNet",  # 使用 GhostFaceNet 模型
        enforce_detection=False  # 关闭强制检测
    )
    
    # 如果提取到的特征非空，则获取嵌入向量
    if face_representation:
        embedding = face_representation[0]['embedding']
        train_emb_list.append(embedding)

# 将所有面部嵌入转换为 tensor
train_emb = torch.tensor(train_emb_list)

# 调用 evaluate 函数评估模型的性能
face_score, clip_score, mis = evaluate(
    pretrained_model_name_or_path=pretrained_model_name_or_path,  # 预训练模型的名称或路径
    weight_dtype=weight_dtype,  # 模型的权重数据类型
    seed=seed,  # 随机数种子，确保可重复的推理结果
    unet_path=unet_path,  # 训练保存的 UNet 模型的权重文件路径
    text_encoder_path=text_encoder_path,  # 训练保存的文本编码器模型的权重文件路径
    validation_prompt=validation_prompt,  # 验证时使用的文本提示，输入模型生成图像
    output_folder=inference_path,  # 推理时生成的图像保存到指定的文件夹
    train_emb=train_emb  # 使用的训练图像的面部特征嵌入，用于评估面部相似度
)

# 打印评估结果
print(f"面部相似度评分 (平均欧氏距离): {face_score:.4f} (越低越好，表示生成图像与训练图像更相似)")
print(f"CLIP 评分 (平均相似度): {clip_score:.4f} (越高越好，表示生成图像与文本提示的相关性更强)")
print(f"无面部图像数量: {mis} (无法检测到面部的生成图像数量)")